## 빅데이터 실습
### 타이타닉 생존자 상관분석
- 상관계수로 두 변수간의 상관관계 유추
  - correlation coefficient
- 상관계수 값(-1~0~1) 
  - 음수면 서로간의 음의 상관관계(한쪽값이 올라가면, 다른값은 떨어짐)
  - 양수면 서로간의 양의 상관관계(한쪽값이 올라가면, 다른값도 올라감)
  - 0.0 ~ 0.2: 상관관계가 거의 없다
  - 0.2 ~ 0.4: 약간 상관관계가 있다
  - 0.4 ~ 0.6: 상관관계가 있다
  - 0.6 ~ 0.8: 강한 상관관계가 있다
  - 0.8 ~ 1.0: 서론간 매우 긴밀한 연관이 있다. 상관관계 최고
### 데이터 수집
#### Seaborn 테스트데이터

In [ ]:
## 필수 라이브러리 사용등록
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 한글 깨짐 문제 해결
from matplotlib import rcParams, font_manager, rc

font_path = 'C:/Windows/Fonts/NanumGothicCoding.ttf' # 나눔고딕코딩 사용, 나눔고딕에서 오류발생(!)
font = font_manager.FontProperties(fname=font_path).get_name() # 실제 설치된 폰트 이름조회
rc('font', family=font) # 한글깨짐현상 해결!!
rcParams['axes.unicode_minus'] = False # 한글 사용시 마이너스 표시 깨짐 해결!

In [ ]:
sns.set_style('darkgrid')

In [ ]:
# seaborn 타이타닉 데이터 
dfTitanic = sns.load_dataset('titanic')
dfTitanic

#### 타이타닉 생존자 데이터 문제
- 결측치가 너무 많다 -> 결측치 제거 필요


In [ ]:
dfTitanic.isnull().sum()

In [ ]:
dfTitanic.info()

In [ ]:
dfTitanic.describe()

In [ ]:
# 조사결과 나이대는 20대 중후반이 가장 많음
ageMedian = dfTitanic['age'].median()

In [ ]:
# 나이에 빈값을 중앙값으로 채워넣음
dfTitanic['age'] = dfTitanic['age'].fillna(ageMedian)

In [ ]:
dfTitanic.info()

In [ ]:
# 정박항 결측치 입력, 대부분 S(Southampton)에서 배를 탔음
dfTitanic['embarked'].value_counts()

In [ ]:
dfTitanic['embark_town'].value_counts()

In [ ]:
dfTitanic['embarked'] = dfTitanic['embarked'].fillna('S')

In [ ]:
dfTitanic['embark_town'] = dfTitanic['embark_town'].fillna('Southampton')

In [ ]:
dfTitanic.info()

In [ ]:
## 데크 결측치 제거
dfTitanic['deck'].value_counts()

In [ ]:

## 데크가 제일 많은 빈도로 나온것 C데크 
## 각 데크별로 빈도의 차이가 별로 없기때문에 
## C데크를 입력, 데크로 상관관계 비교는 지표에서는 사용하지 않는 것이 좋음
dfTitanic['deck'] = dfTitanic['deck'].fillna('C')

In [ ]:
dfTitanic.info()

In [ ]:

dfTitanic.to_excel('./data/타이타닉생존자_전처리완.xlsx', index=False)

In [ ]:

dfTitanic['survived'][dfTitanic['sex'] == 'male'].value_counts()

In [ ]:
# 파이차트
f, ax = plt.subplots(1, 2, figsize=(12, 6))

# 판다스 DF, 시리즈는 matplotlib 차트로 그릴 수 있음
maleLabels= ['사망', '생존']
femaleLabels = ['생존', '사망']
maleColors = ['gray', 'lightGreen']
femaleColors = ['lightGreen', 'gray']
wedgeProps = { 'width': 0.7, 'edgecolor': 'w', 'linewidth': 5 }

dfTitanic['survived'][dfTitanic['sex'] == 'male'].value_counts().plot.pie(ax=ax[0], 
                                                    shadow=False, explode=[0, 0.01], autopct='%.1f%%',
                                                    labels=maleLabels, colors=maleColors, wedgeprops=wedgeProps)
dfTitanic['survived'][dfTitanic['sex'] == 'female'].value_counts().plot.pie(ax=ax[1], 
                                                    shadow=False, explode=[0, 0.01], autopct='%.1f%%',
                                                    labels=femaleLabels, colors=femaleColors, wedgeprops=wedgeProps)

ax[0].set_title('타이타닉 생존율(남성)')
ax[1].set_title('타이타닉 생존율(여성)')
ax[0].set_axis_off() # 축의 count 라벨 필요없음
ax[1].set_axis_off() # 축의 count 라벨 필요없음

plt.show()

In [ ]:
## 등급(pclass) 생존자 수 바차트
plt.figure(figsize=(10, 5))
# stat= count, percent 등이 있음

ax = sns.countplot(dfTitanic, x='pclass', hue='survived', stat='count')

for p in ax.patches: # 바 갯수만큼 반복
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2., height+5, int(height), ha='center', size=10)

plt.show()

##### 데이터 속성(변수)간 상관관계
- pandas DF에 corr()함수 사용

In [ ]:
# 상관계수는 object, category는 적용불가
dfTitanic.info()

In [ ]:
dfTitanic.head()

In [ ]:
# 상관계수 메서드: pearson, kendall, spearman
# 상관관계를 모든 속성으로 다 집계
dfTitanic[['survived', 'pclass', 'age', 'sibsp', 'parch', 'fare', 'adult_male', 'alone']].corr(method='pearson')

In [ ]:
# 각 두가지 속성끼리 상관관계 도출
dfTitanic['survived'].corr(dfTitanic['fare'])

In [ ]:
sns.pairplot(dfTitanic, hue='survived')
plt.show()

In [ ]:
dfTitanic = pd.read_excel('./data/타이타닉생존자_전처리완.xlsx')

In [ ]:
## 상관관계를 히트맵 시각화
def categoryAge(x):
    if x < 10: return 0
    elif x < 20: return 1
    elif x < 30: return 2
    elif x < 40: return 3
    elif x < 50: return 4
    elif x < 60: return 5
    elif x < 70: return 6
    elif x < 80: return 7
    else: return 8

In [ ]:
# 히트맵에서 사용하기 위해서 컬럼 재생산
dfTitanic['age2'] = dfTitanic['age'].apply(categoryAge) # 컬럼의 값을 함수를 적용해서 값을 낼 수 있음
dfTitanic['sex'] = dfTitanic['sex'].map({'male':1, 'female':0}) # 여러번 수행하면 안됨
dfTitanic['family'] = dfTitanic['sibsp'] + dfTitanic['parch'] + 1

In [ ]:
dfTitanic.tail()

In [ ]:
## 히트맵
plt.figure(figsize=(12, 8)) # 차트 크기를 지정
hData = dfTitanic[['survived', 'sex', 'age2', 'family', 'pclass', 'fare']]
## 속성정리, square=각 히트탭 셀 정사각형
## annot=True 값 화면표시
sns.heatmap(hData.astype(float).corr(), linewidths=1.0, square=False, vmax=1.0, cmap='YlGn', linecolor='white', annot=True, annot_kws={'size': 10 })

plt.title('타이타닉 속성별 상관관계 히트맵')
plt.show()

- 상관관계는 음의관계, 양의관계 모두 중요. 절대값 0.4~1.0 사이의 값에 대한 속성간의 분석을 시작하면 됨
- 0.2 이하의 데이터는 분석대상에서 제외